In [19]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from random import randint
from sklearn import random_projection
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances


In [20]:
#read files
with open('train.dat',"r") as fh:
    lines = fh.readlines()
    data = [l.split() for l in lines]
    data = [[int(i) for i in li]for li in data ]

In [21]:
doc=[li[::2]for li in data]

In [22]:
value = [li[1::2] for li in data]
count=[i for li in value for i in li]

In [23]:
indptr = [0]
for d in doc:
    n = len(d) + indptr[-1]
    indptr.append(n)
    
indices = np.array([item for i in doc for item in i])
mat = csr_matrix((count, indices, indptr), shape =(8580,max(indices)))

In [24]:
#random projection
transformer = random_projection.SparseRandomProjection()
mat_rd = transformer.fit_transform(mat)


In [25]:
mat_rd_l2 = normalize(mat_rd, norm='l2', axis =1, copy= True)

In [26]:
def randomKdocInd(k):
    centroid = [randint(0,8579) for i in range (0,k)]
    return centroid

Kcent = randomKdocInd(7)

In [27]:
centroid_mat = mat_rd_l2[Kcent].toarray()

In [28]:
cen_initial = centroid_mat

In [29]:
clusters = {}

In [30]:
def kMean(mat_rd_l2, centroid_mat):
    #calculate cosine sim
    sim = cosine_similarity(centroid_mat, mat_rd_l2)
    placement = np.argmax(sim,axis = 0)
    #assignCluster
    assignCluster(placement, mat_rd_l2)
    #recalculate centroid
    c_mat = recalCentroid(centroid_mat,mat_rd_l2)
    
    return c_mat, placement

In [31]:
def findbestcentroids(mat_rd_l2,centroid_mat):
    while True:
        centroid_mat_new,res = kMean(mat_rd_l2, centroid_mat)
        if np.allclose(centroid_mat,centroid_mat_new):
            return res,centroid_mat,centroid_mat_new
        else:
            centroid_mat = centroid_mat_new

In [32]:
#calculate cosine sim and cluter each doc
def assignCluster(placement, mat_rd_l2):
    for c in range(0,7):
        clusters[c] = []
        
    for i,j in enumerate(placement):
        clusters[j].append(i)

In [33]:
#recalculate centroid for 7 clusters
def recalCentroid(centroid_mat,mat_rd_l2):
    cent_mat = []
    for c in range(0,7):
        mat_c =  mat_rd_l2[clusters[c]]
        centroid_new = np.squeeze(np.asarray(mat_c.mean(axis=0)))
        cent_mat.append(centroid_new)
        
    return cent_mat
    

In [34]:
def writeToFile(res,filename):
    resFile = open(filename, 'w')
    for i in range (len(res)):
        resFile.write(str(res[i]) +'\n')

In [35]:
res,c,cn = findbestcentroids(mat_rd_l2,centroid_mat)

In [36]:
writeToFile(res,"res9.dat")